In [1]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up Kaggle API credentials
kaggle_json_path = "C:/Users/badal/.kaggle/kaggle.json" # Update this path
os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(kaggle_json_path)

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Define the dataset name and download path
dataset_name = 'kazanova/sentiment140'
download_path = './sentiment140'  # Directory where the dataset will be downloaded

# Create the download path directory if it doesn't exist
if not os.path.exists(download_path):
    os.makedirs(download_path)

# Download and unzip the dataset
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

print(f"Dataset downloaded and extracted to {download_path}")


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
Dataset downloaded and extracted to ./sentiment140


In [3]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\badal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
column_names = ['target','id','date','flag','user','text']
df = pd.read_csv('C:/Users/badal/Documents/Project/Tweeter Sentiment Analysis/sentiment140/twitterdata.csv',names=column_names,encoding='ISO-8859-1')

In [7]:
df.shape

(1600000, 6)

In [8]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [10]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [11]:
df.replace({'target': {4: 1}}, inplace=True)

In [12]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [13]:
stemer = PorterStemmer()

In [14]:
def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

In [15]:
df['stemmed_content'] = df['text'].apply(stemming)

In [17]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [18]:
df['stemmed_content']

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object

In [19]:
df['stemmed_content'][1]

'upset updat facebook text might cri result school today also blah'

In [20]:
df['target']

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

In [21]:
X = df['stemmed_content'].values
y = df['target'].values

In [23]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [24]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [30]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [31]:
x_train.shape

(1280000,)

In [32]:
x_test.shape

(320000,)

In [34]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [35]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [37]:
print(x_train[0])

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384


In [38]:
model = LogisticRegression(max_iter=1000)

In [39]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [40]:
x_train_pred = model.predict(x_train)
train_accuracy = accuracy_score(y_train,x_train_pred)

In [41]:
train_accuracy

0.8101859375

In [42]:
x_test_pred = model.predict(x_test)
test_accuracy = accuracy_score(y_test,x_test_pred)

In [43]:
test_accuracy

0.7780125

In [44]:
import pickle


In [45]:
filename = 'trained_model.pkl'
pickle.dump(model,open(filename,'wb'))

In [47]:
loaded_model = pickle.load(open('C:/Users/badal/Documents/Project/Tweeter Sentiment Analysis/trained_model.pkl','rb'))

In [49]:
x_instance = x_test[100]
print(y_test[100])

pred = loaded_model.predict(x_instance)
print(pred)
if pred[0] == 0:
    print('Negative tweet')
else:
    print('Positive tweet')


0
[0]
Negative tweet


In [50]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB()
model2.fit(x_train,y_train)

MultinomialNB()

In [51]:
pred = model2.predict(x_test)

In [52]:
print(accuracy_score(pred,y_test))

0.755815625


In [53]:
training_pred = model2.predict(x_train)

In [54]:
print(accuracy_score(training_pred,y_train))

0.8200265625


In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X = df['stemmed_content'].values
y = df['target'].values

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

# Ensure the data is in the correct format for the Tokenizer
x_train = x_train.tolist()
x_test = x_test.tolist()

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

# Convert text to sequences
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Pad sequences to ensure uniform input length
max_len = 100
x_train_pad = pad_sequences(x_train_seq, maxlen=max_len)
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len)

# Define the model
model3 = Sequential()
model3.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
model3.add(SpatialDropout1D(0.2))
model3.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model3.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model3.add(Dense(1, activation='sigmoid'))

# Compile the model
model3.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the model
history = model3.fit(x_train_pad, y_train, epochs=10, batch_size=512, validation_data=(x_test_pad, y_test), verbose=1, callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
test_loss, test_accuracy = model3.evaluate(x_test_pad, y_test, verbose=0)
print(f'Test Accuracy: {test_accuracy}')